<a href="https://colab.research.google.com/github/qw-4735/PyTorch/blob/main/%EC%8B%A0%EA%B2%BD%EB%A7%9D_(_PyTorch_tutorials).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 신경망(Neural Networks)
- 신경망은 torch.nn 패키지를 사용하여 생성
- nn 은 모델을 정의하고 미분하는데 autograd를 사용
- nn.Module은 계층(layer)과 output을 반환하는 forward 메서드를 포함

torch.nn은 미니배치만 지원한다.
 
즉, torch.nn 패키지 전체는 하나의 샘플이 아닌, 샘플들의 미니배치만을 입력으로 받는다.

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [6]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    # 컨볼루션 커널 정의
    self.conv1 = nn.Conv2d(1,6,5)  # 입력 이미지 채널 1개, 출력채널 6개, 5x5 정사각 컨볼루션 행렬
    self.conv2 = nn.Conv2d(6,16,5)
    
    self.fc1 = nn.Linear(16*5*5 ,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)

  def forward(self,x):
    x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))    # (2,2) 크기 윈도우에 대해 max pooling
    x = F.max_pool2d(F.relu(self.conv2(x)),2)        # 크기가 제곱수라면, 하나의 숫자만을 특정
    x = torch.flatten(x,1)    # 배치 차원을 제외한 모든 차원을 하나로 flatten
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x) 
    return x

net = Net()
print(net)     

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


forward 함수만 정의하고 나면, (변화도를 계산하는) backward 함수는 autograd를 사용하여 자동으로 정의된다.

In [7]:
# 모델의 학습 가능한 매개변수들은 net.parameters()에 의해 반환됨
params = list(net.parameters())
print(len(params))
print(params[0].size())   # conv1의 weight

10
torch.Size([6, 1, 5, 5])


In [8]:
# 임의의 32 x 32 입력값을 넣어보자
input = torch.randn(1,1,32,32)
out = net(input)
print(out)

tensor([[ 0.0886,  0.0885,  0.0524,  0.0623,  0.0221,  0.0443, -0.1012,  0.0451,
          0.0299,  0.0222]], grad_fn=<AddmmBackward0>)


In [9]:
# 모든 매개변수의 변화도 버퍼(gradient buffer)를 0으로 설정하고, 무작위 값으로 역전파
net.zero_grad()
out.backward(torch.randn(1,10))

### 손실함수 (Loss Function)
- 손실함수는 (output, target)을 한 쌍(pair)의 입력으로 받아, 출력(output)이 정답(target)으로부터 얼마나 멀리 떨어져있는지 추정하는 값을 계산
- nn 패키지에는 여러가지 손실함수들이 존재 ex) nn.MSEloss

In [ ]:
output = net(input)
target = torch.randn(10)  # 예시를 위한 임의의 정답
target = target.view(1,-1)   # 출력과 같은 shape를 만듦
criterion = nn.MSELoss  

loss = criterion(output, target)
print(loss)

In [ ]:
# 역전파의 몇 과정 예시
print(loss.grad_fn)    # MSELoss
print(loss.grad_fn.next_functions[0][0])   # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])   # ReLU

### 역전파
오차를 역전파하기 위해서는 loss.backward()만 해주면 된다.

기존에 계산된 변화도의 값을 누적시키고 싶지 ㅇㄶ다면, 기존에 계산된 변화도를 0으로 만드는 작업이 필요

In [ ]:
net.zero_grad()   # 모든 매개변수의 변화도 버퍼를 0으로 만듦

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)    # loss.backward()를 호출하여 역전파 후의 conv1의 bias변수의 변화도를 살펴보자

### 가중치 갱신
new weight = weight - learning rate * gradient

In [14]:
learning_rate=0.01
for f in net.parameters():
  f.data.sub_(f.grad.data * learning_rate)

In [ ]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training loop)은 다음과 같다:
optimizer.zero_grad()   # 변화도 버퍼를 0으로
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()  # 업데이트 진행